# JuliaCon 2024
</br>

## End-to-End AI (E2EAI) with Julia, K0s, and Argo Workflow
</br>

##### Presentor: Paulito Palmes, IBM Research
#####      Date: July 11, 2024 

# OUTLINE

- ### The Motivations Behind E2EAI (End-to-End AI)
- ### Components of E2EAI
- ### The Julia AI/ML Solution Use-case
- ### The Future

# The Motivations Behind E2EAI

- current paradigm does not exloit tightly integration of IaC and MLOPs in deploying AI solutions
- issues with no tight integration of IaC and MLOPs in deploying AI solutions
  - difficult to identify optimal infrastructure 
  - difficult to predict resource viability and feasibility
  - difficult to infer the cost 
  - difficult to identify performance bottlenecks and root-cause analysis

# End-to-End AI (E2EAI)
<center><img  src="./yamlcontents.png" width="600"/></center>

- E2EAI is a unified framework tightly integrating MLOps and IaC 
  - single yaml file: Infrastructure + ML Pipeline + LifeCycle Management
  - single yaml file to describe both the IaC and MLOPs
  - yaml workflow templates imply zero to minimal coding
  - collection of yamls become inputs to LLM for intent-driven E2EAI

# Components of E2EAI
<center><img  src="./e2eai-components.png" width="700"/></center>

- SUNRISE-6G
  - SUstainable federatioN of Research Infrastructures \
    for Scaling-up Experimentation in 6G
  - H2020 EU Project (3 years)

# The Julia AI/ML Solution Use-case

- AutoMLPipeline workflow
- Integrating AutoMLPipeline in E2EAI

### Load ML pipeline preprocessing components and models

In [78]:
using AutoMLPipeline;
import PythonCall; const PYC=PythonCall; warnings = PYC.pyimport("warnings"); warnings.filterwarnings("ignore")

#### Decomposition
pca = skoperator("PCA"); fa  = skoperator("FactorAnalysis"); ica = skoperator("FastICA")
#### Scaler 
rb   = skoperator("RobustScaler"); pt   = skoperator("PowerTransformer"); norm = skoperator("Normalizer")
mx   = skoperator("MinMaxScaler"); std  = skoperator("StandardScaler")
#### categorical preprocessing
ohe = OneHotEncoder()
#### Column selector
catf = CatFeatureSelector(); numf = NumFeatureSelector(); disc = CatNumDiscriminator()
#### Learners
rf = skoperator("RandomForestClassifier"); gb = skoperator("GradientBoostingClassifier"); lsvc = skoperator("LinearSVC")
svc = skoperator("SVC"); mlp = skoperator("MLPClassifier")
ada = skoperator("AdaBoostClassifier"); sgd = skoperator("SGDClassifier")
skrf_reg = skoperator("RandomForestRegressor"); skgb_reg = skoperator("GradientBoostingRegressor")
jrf = RandomForest(); tree = PrunedTree()
vote = VoteEnsemble(); stack = StackEnsemble(); best = BestLearner();

### Prepare dataset for classification

In [79]:
# Make sure that the input feature is a dataframe and the target output is a 1-D vector.
using AutoMLPipeline
profbdata = getprofb()
X = profbdata[:,2:end] 
Y = profbdata[:,1] |> Vector;
head(x)=first(x,10)
head(profbdata)

Row,Home.Away,Favorite_Points,Underdog_Points,Pointspread,Favorite_Name,Underdog_name,Year
,String7,Int64,Int64,Float64,String3,String3,Int64
1,away,27,24,4.0,BUF,MIA,89
2,at_home,17,14,3.0,CHI,CIN,89
3,away,51,0,2.5,CLE,PIT,89
4,at_home,28,0,5.5,NO,DAL,89
5,at_home,38,7,5.5,MIN,HOU,89
6,at_home,34,20,6.0,DEN,KC,89
7,away,31,21,6.0,LAN,ATL,89
8,at_home,24,27,2.5,NYJ,NE,89
9,away,16,13,1.5,PHX,DET,89


### Pipeline to transform non-numeric features to one-hot encoding

In [80]:
pohe = catf |> ohe
tr = fit_transform!(pohe,X,Y)
head(tr)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Sample pipeline to transform numerical features to pca and ica and concatenate them

In [81]:
pdec = (numf |> pca) + (numf |> ica)
tr = fit_transform!(pdec,X,Y)
head(tr)

Row,x1,x2,x3,x4,x1_1,x2_1,x3_1,x4_1
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2.47477,7.87074,-1.10495,0.902431,1.21195,0.433646,-0.695915,-0.0803511
2,-5.47113,-3.82946,-2.08342,1.00524,1.16887,-0.851005,-0.178678,0.566579
3,30.4068,-10.8073,-6.12339,0.883938,1.1001,-1.91135,-1.29918,-2.99346
4,8.18372,-15.507,-1.43203,1.08255,1.18419,-1.70322,0.49884,-0.955535
5,16.6176,-6.68636,-1.66597,0.978243,1.19617,-0.881656,-0.179992,-1.66503
6,10.2588,5.22112,0.0731649,0.928496,1.24395,0.448154,-0.327303,-0.910133
7,7.13435,5.60902,0.368661,0.939797,1.24838,0.511012,-0.231967,-0.601939
8,-1.16369,10.3011,-2.15564,0.86957,1.18657,0.450458,-1.07008,0.333907
9,-6.38764,-4.92017,-3.57339,0.986345,1.13191,-1.20871,-0.499935,0.672852


### Another example of more complex pipeline

In [82]:
ppt = (numf |> rb |> ica) + (numf |> pt |> pca)
tr = fit_transform!(ppt,X,Y)
head(tr)

Row,x1,x2,x3,x4,x1_1,x2_1,x3_1,x4_1
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.21199,0.43535,-0.694697,-0.0810005,-0.64552,1.40289,-0.0284468,0.111773
2,-1.16928,-0.85302,-0.181524,0.56178,-0.832404,0.475629,-1.14881,-0.01702
3,-1.10209,-1.89464,-1.28761,-3.00831,1.54491,1.65258,-1.35967,-2.57866
4,-1.18506,-1.69834,0.501783,-0.961586,1.32065,0.563565,-2.05839,-0.74898
5,-1.19699,-0.872598,-0.173169,-1.66994,1.1223,1.45555,-0.88864,-0.776195
6,-1.24399,0.453625,-0.32246,-0.909088,0.277462,1.70936,0.00130938,0.0768767
7,-1.2483,0.514863,-0.228402,-0.600174,0.0977821,1.58007,-0.0364638,0.258464
8,-1.18667,0.450464,-1.07065,0.331705,-1.31815,1.27463,0.00789964,-0.0553192
9,-1.1326,-1.21095,-0.503596,0.664878,-1.29056,0.326316,-1.31916,-0.511818


### Evaluating complex preprocessing pipeline together with RandomForest learner

In [87]:
prf = (catf |> ohe) + (numf |> rb |> fa) + (numf |> pt |> pca) |> rf
crossvalidate(prf,X,Y,"accuracy_score")

fold: 1, 0.6567164179104478
fold: 2, 0.8059701492537313
fold: 3, 0.6029411764705882
fold: 4, 0.6268656716417911
fold: 5, 0.6268656716417911
fold: 6, 0.7014925373134329
fold: 7, 0.6567164179104478
fold: 8, 0.7352941176470589
fold: 9, 0.7014925373134329
fold: 10, 0.7164179104477612
errors: 0


(mean = 0.6830772607550484, std = 0.06123739916394872, folds = 10, errors = 0)

### Evaluating complex preprocessing pipeline together with Linear SVM learner

In [88]:
plsvc = ((numf |> rb |> pca)+(numf |> rb |> fa)+(numf |> rb |> ica)+(catf |> ohe )) |> lsvc
crossvalidate(plsvc,X,Y,"accuracy_score")

fold: 1, 0.6716417910447762
fold: 2, 0.7611940298507462
fold: 3, 0.8382352941176471
fold: 4, 0.7313432835820896
fold: 5, 0.8059701492537313
fold: 6, 0.6865671641791045
fold: 7, 0.6417910447761194
fold: 8, 0.7647058823529411
fold: 9, 0.6567164179104478
fold: 10, 0.7313432835820896
errors: 0


(mean = 0.7289508340649693, std = 0.06506398947193438, folds = 10, errors = 0)

### Parallel search of the best ML pipeline

In [85]:
using Random, DataFrames, Distributed
nprocs() == 1 && addprocs()
@everywhere using DataFrames; @everywhere using AutoMLPipeline
@everywhere begin
    import PythonCall; const PYC=PythonCall; warnings = PYC.pyimport("warnings"); warnings.filterwarnings("ignore")
end
@everywhere begin
  profbdata = getprofb(); X = profbdata[:,2:end]; Y = profbdata[:,1] |> Vector;
end
@everywhere begin
  jrf  = RandomForest(); ohe  = OneHotEncoder(); catf = CatFeatureSelector(); numf = NumFeatureSelector()
  tree = PrunedTree(); ada  = skoperator("AdaBoostClassifier"); disc = CatNumDiscriminator()
  sgd  = skoperator("SGDClassifier"); std  = skoperator("StandardScaler"); lsvc = skoperator("LinearSVC")
end

learners = @sync @distributed (vcat) for learner in [jrf,ada,sgd,lsvc,tree]
   pcmc = disc |> ((catf |> ohe) + (numf |> std)) |> learner
   println(learner.name[1:end-4])
   mean,sd,_ = crossvalidate(pcmc,X,Y,"accuracy_score",3)
   DataFrame(name=learner.name[1:end-4],mean=mean,sd=sd)
end;

      From worker 2:	rf
      From worker 3:	AdaBoostClassifier
      From worker 4:	SGDClassifier
      From worker 5:	LinearSVC
      From worker 6:	prunetree
      From worker 2:	fold: 1, 0.6964285714285714
      From worker 6:	fold: 1, 0.5446428571428571
      From worker 4:	fold: 1, 0.6607142857142857
      From worker 5:	fold: 1, 0.7410714285714286
      From worker 2:	fold: 2, 0.6607142857142857
      From worker 6:	fold: 2, 0.6205357142857143
      From worker 3:	fold: 1, 0.7008928571428571
      From worker 4:	fold: 2, 0.6785714285714286
      From worker 5:	fold: 2, 0.6830357142857143
      From worker 2:	fold: 3, 0.6964285714285714
      From worker 2:	errors: 0
      From worker 6:	fold: 3, 0.5803571428571429
      From worker 6:	errors: 0
      From worker 4:	fold: 3, 0.7008928571428571
      From worker 4:	errors: 0
      From worker 5:	fold: 3, 0.7142857142857143
      From worker 5:	errors: 0
      From worker 3:	fold: 2, 0.7276785714285714
      From worker 3:	fold: 3,

### Best Pipeline

In [86]:
@show sort!(learners,:mean,rev=true);

sort!(learners, :mean, rev = true) = 5×3 DataFrame
 Row │ name                mean      sd
     │ String              Float64   Float64
─────┼─────────────────────────────────────────
   1 │ LinearSVC           0.712798  0.0290465
   2 │ AdaBoostClassifier  0.702381  0.0245874
   3 │ rf                  0.684524  0.0206197
   4 │ SGDClassifier       0.68006   0.0201306
   5 │ prunetree           0.581845  0.0379683


# E2EAI Application

## Infrastructure Creation Automation

<center><img src="./k0s.png" width="500"/></center>

<center><img src="./cluster.png" width="800"/></center>

## AI as a Service: Zero Coding Using Workflow Template
<center><img src="./template.png" width="1000"/></center>

## Explicit ML Pipeline

<center><img src="./mlpipeline.png" width="600"/></center>

## Optimal Pipeline Discovery by AutoML
<center><img src="./lowcode.png" width="500"/></center>

## Low vs High Pipeline Complexity
<center><img src="./low-high-comp.png" width="300"/></center>

### Low Complexity Pipeline
<center><img src="./low-comp.png" width="500"/></center>

### High Complexity Pipeline 
<center><img src="./high-comp.png" width="500"/></center>